In [1]:
%load_ext autoreload
%autoreload 2

# DATA MANIPULATION
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import numpy as np

# DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns

# VIEWING OPTIONS IN THE NOTEBOOK
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import LabelEncoder


from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.impute import KNNImputer
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('drive/MyDrive/merge_iex_censo_banco.csv')
#df = pd.read_csv('drive/MyDrive/merge_iex_censo_banco2015-2021.csv')


In [ ]:
df.isna().sum()

In [ ]:
df = df.drop(columns=['CO_ESCOLA', 'CO_MUNICIPIO', 'CNPJ', 'Unnamed: 0'])
df.tail()

In [ ]:
###Usar uma amostra do DF
#df = df.sample(200000)
df_2021=df[df['ANO']==2016]
df=df[df['ANO']==2015]

#select data for 2015 and 2016
df.drop_duplicates(inplace=True)


In [ ]:
df['IEX']

In [ ]:
# preprocessing
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')

df['SG_UF'] = df['SG_UF'].astype('category')

preprocessor = make_column_transformer(
    (num_transformer, ['POP']),
    (cat_transformer,['SG_UF']),
    remainder='passthrough'
)

preprocessor

In [ ]:
df

In [ ]:
df_2021 = df_2021.sample(199014)
print(df.shape,df_2021.shape)

In [ ]:
from pandas.core.nanops import na_accum_func
from sklearn.model_selection import train_test_split
y = df['IEX']
X = df.drop(columns=['IEX'])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)
#X_tran_2021,X_test_2021,y_train_2021,y_test_2021 = train_test_split(df_2021.drop(columns=['IEX']),df_2021['IEX'],test_size = 0.2, random_state = 42)

In [ ]:
y_train

In [ ]:
X_train

In [ ]:
# não usary_train = to_categorical(y_train)

In [ ]:
# train_t


In [ ]:
X_train.shape

In [ ]:
# y_train.shape

In [ ]:
y_pred = np.mean(y_train)
print(f'The mean prediction is {y_pred:.3f}')

absolute_error_test_baseline = [np.abs(y_true - y_pred) for y_true in y_test]
mean_absolute_error_test_baseline = np.mean(absolute_error_test_baseline)

print(f'The benchmark mean absolute error is of {mean_absolute_error_test_baseline:.3f}')

In [ ]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

def initialize_model():

    #################################
    #  1 - Model architecture       #
    #################################
    
    model = Sequential()
    
    # Input Layer
    model.add(layers.Dense(64, activation='relu')) #, input_shape = X.shape[1:]))
    
    # Hidden Layers
    model.add(layers.Dense(32, activation='relu'))
    layers.Dropout(0.2)
    model.add(layers.Dense(16, activation='relu'))
    layers.Dropout(0.2)
    model.add(layers.Dense(8, activation='relu'))

    # Predictive Layer
    model.add(layers.Dense(1, activation='linear')) #linear
    
    ##################################
    #  2 - Our recommended compiler  #
    ##################################
    
    adam_opt = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999) 
    model.compile(optimizer=adam_opt,
                  loss='mse', 
                  metrics = ['mae'])      #metrics = ['accuracy']), usar em caso de softmax
    
    return model





In [ ]:
from pandas.core.missing import check_value_size
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
es = callbacks.EarlyStopping(patience=20, restore_best_weights=True)
clf = KerasRegressor(initialize_model, nb_epoch=300, batch_size=16, verbose=0, callbacks=[es],validation_split=0.2)
#estimators.append(('clf', Keras(build_fn=create_baseline, nb_epoch=300, batch_size=16, verbose=0, callbacks=[...your_callbacks...])))

pipeline = make_pipeline(preprocessor,clf)
pipeline

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import cross_validate

result = cross_validate(pipeline,X_test,y_test, scoring = ['neg_mean_absolute_error'])
result

In [ ]:
oeuaoeua  

#Stop here

In [ ]:
history = pipeline.fit(X_train, 
                    y_train,
                    validation_split =  0.2,
                    epochs = epochs,         # Play with this until your validation loss overfit
                    batch_size = batch_size, # Let's keep a small batch size for faster iterations
                    callbacks = [es],
                    verbose = 0)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import callbacks
es = callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [ ]:
# 1. Initializing a NeuralNet with its architecture and its compilation method
model = initialize_model(X_train)
model.summary()

In [ ]:
# 2. Training the model
epochs = 500 #500 e0.2 overfitou
batch_size = 16

history = model.fit(train_t, 
                    y_train,
                    validation_split =  0.2,
                    epochs = epochs,         # Play with this until your validation loss overfit
                    batch_size = batch_size, # Let's keep a small batch size for faster iterations
                    callbacks = [es],
                    verbose = 0)

In [ ]:
# 3. Evaluating the model = mudar para modelo de classificação
model.evaluate(test_t, y_test) #, modelo softmax

In [ ]:
def plot_history(history):
    plt.plot(np.sqrt(history.history['loss']))
    plt.plot(np.sqrt(history.history['val_loss']))
    plt.title('Model Loss')
    plt.ylabel('mae')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='best')
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
#### Fazer predições
predictions = model.predict(X_test_preproc)
predictions

In [ ]:
# Salvar resultados
# results.to_csv("submission_final.csv", header = True, index = False)